In [93]:
# email2inventory
# read evergreen email
# this script will fail if scripts/email2inventory/data/evergreen-20200930.txt doesn't exist

import os
import re

def evergreen_read(filename):
    date = ""
    product_flag = False
    position = 0
    items = {}
    f = open(filename)
    lines = f.readlines()
    while not product_flag:
        line = lines.pop(0).strip()
        m = re.match(r"Date: (\w* \d+, \d{4})", line)
        if m:
            date = m.group(1)
        if line == "Product":
            product_flag = True
    line = lines.pop(0).strip()
    assert line == "Quantity"
    line = lines.pop(0).strip()
    assert line == "Price"
    while product_flag:
        m = re.match(r"(.*) - 1 Week", lines.pop(0).strip())
        try:
            itemname = m.group(1)
        except:
            break
        quantity = lines.pop(0).strip()
        price = lines.pop(0).strip()
        items[itemname] = [quantity,price]
    f.close()
    return({'items': items, 'date' : date})
    
print(evergreen_read(os.path.join("email_data/evergreen-20200930.txt")))

{'items': {"Red's Best Hake": ['2', '$32.00']}, 'date': 'September 30, 2020'}


In [102]:
# email2inventory
# read kistler email
# this script will fail if scripts/email2inventory/data/kistler-20200912.txt doesn't exist

import os
import re

def kistler_read(filename):
    date = ""
    product_flag = False
    position = 0
    items = {}
    f = open(filename)
    lines = f.readlines()
    while not product_flag and lines:
        line = lines.pop(0).strip()
        m = re.match(r"Date: (\w* \d+, \d{4})", line)
        if m:
            date = m.group(1)
        elif line == "Total":
            product_flag = True
        while product_flag:
            m = re.search(r"(\d+)-Bottle Mixed Package\*$", lines.pop(0).strip())
            items = get_package(int(m.group(1)),lines)
            product_flag = False
    f.close()
    return({'items' : items, 'date' : date})

def get_package(bottles,lines):
    inventory = {}
    while bottles > 0:
        line = lines.pop(0).strip()
        m = re.match(r"\•(\d+) btls (\d{4}) (.*)$",line)
        inventory[m.group(3)] = [int(m.group(1)), m.group(2)]
        bottles -= int(m.group(1))
    return inventory
    
print(kistler_read(os.path.join("email_data/kistler-20200912.txt")))

{'items': {'McCrea Vineyard Chardonnay Sonoma Mountain': [2, '2018'], 'Durell Vineyard Chardonnay Sonoma Coast': [2, '2018'], 'Laguna Ridge Vineyard Pinot Noir Russian River Valley': [2, '2018']}, 'date': 'September 12, 2020'}


This uses OAUTH2 to get the two folders from the Sortly API and then open an item and get the image.

Boris figured out how to do it using `urllib.request`, but I think I should be able to do it using `requests`.  Also, I think we're doing some sort of brute force OAUTH and that there is something more complicated but proper - maybe using requests. Something to try to figure out.

I moved the client key to sortly_key.sh as a file that prints the key. I ignored it in GitHub and am not syncing it.

In [111]:
# trying to access Sortly API
import urllib.request
import json
import urllib
import requests
import io
import subprocess
from PIL import Image

client_secret = subprocess.check_output("./sortly_key.sh").strip()

headers = {
  'Accept': 'application/json',
  'Authorization': client_secret
}
# trying to use request instead of urllib but can't figure it out
# request = requests.get('https://api.sortly.co/api/v1/items?per_page=&page=&folder_id=&include=', headers)
# print(request)
# response_py = request.json()
request = urllib.request.Request('https://api.sortly.co/api/v1/items?per_page=&page=&folder_id=&include=', headers=headers)
response_body = urllib.request.urlopen(request).read()
response_py = json.loads(response_body)
print(response_py)
print(response_py['data'][0]['name'])
next_url = response_py['meta']['next_page_url']
request = urllib.request.Request(next_url, headers=headers)
response_body = urllib.request.urlopen(request).read()
response_py = json.loads(response_body)
print(response_py['data'][0]['name'])
next_url = response_py['meta']['next_page_url']
request = urllib.request.Request('https://api.sortly.co/api/v1/items/17481596?include=custom_attributes%2Cphotos', headers=headers)
response_body = urllib.request.urlopen(request).read()
response_py = json.loads(response_body)
print(response_py)
photo_url = response_py['data']['photos']['url']
response = requests.get(photo_url)
img = Image.open(io.BytesIO(response.content))
img.show()

{'meta': {'page': 1, 'next_page_url': 'https://api.sortly.co/api/v1/items?folder_id=&include=&page=2&per_page=', 'total_pages': 2, 'total_count': 2}, 'data': [{'id': 17356161, 'name': 'Food', 'price': None, 'quantity': None, 'min_quantity': None, 'notes': None, 'parent_id': None, 'sid': 'S3YNFR0001', 'label_url': None, 'label_url_type': None, 'label_url_extra': None, 'label_url_extra_type': None, 'tag_names': [], 'type': 'folder', 'created_at': '2020-09-30T13:32:44.867Z', 'updated_at': '2020-09-30T13:32:44.867Z'}]}
Food
Durable Things
{'data': {'id': 17481596, 'name': 'Open qPCR 1x Ch Real-Time PCR Machine', 'price': '5799.0', 'quantity': 1, 'min_quantity': None, 'notes': 'Purchased by CCI', 'parent_id': 17444508, 'sid': 'S3YNFT0027', 'label_url': None, 'label_url_type': None, 'label_url_extra': None, 'label_url_extra_type': None, 'tag_names': [], 'type': 'item', 'created_at': '2020-10-04T11:44:23.236Z', 'updated_at': '2020-10-04T12:12:17.159Z', 'custom_attribute_values': [{'value': '2

TypeError: list indices must be integers or slices, not str